In [50]:
import psycopg2
import pandas as pd

db_params = {'database' : 'postgres',
             'user' : 'postgres',
             'password' : 'admin',
             'host' : '127.0.0.1',
             'port' : '5432'}

conn = psycopg2.connect(database=db_params['database'], 
						user=db_params['user'],
                        password=db_params['password'], 
						host=db_params['host'],
                        port=db_params['port']
) 

cur = conn.cursor()

In [ ]:
cur.execute("ALTER TABLE branches ADD is_storage BOOLEAN DEFAULT FALSE;")
cur.execute("UPDATE branches SET is_storage=TRUE WHERE branch_name LIKE '%склад%' OR branch_name LIKE '%Склад%';")

conn.commit() 

In [51]:
cur.execute("ALTER TABLE branches ADD top_products VARCHAR[];")
conn.commit() 

In [53]:
cur.execute('''WITH RankedProducts AS (
  SELECT
    products.product_name,
    sales.sale_quantity,
    branches.branch_name,
    ROW_NUMBER() OVER (PARTITION BY branches.branch_key ORDER BY sales.sale_quantity DESC) AS ranking
  FROM
    sales
    JOIN products ON sales.product_key = products.product_key
    JOIN branches ON sales.branch_key = branches.branch_key
  WHERE 
      (products.product_name NOT LIKE '%Доставка%' AND products.product_name NOT LIKE '%доставка%' AND products.product_name NOT LIKE '%Обработка%')
)
UPDATE branches
SET top_products = (
  SELECT ARRAY_AGG(RankedProducts.product_name) AS top_products
  FROM RankedProducts
  WHERE RankedProducts.branch_name = branches.branch_name AND ranking BETWEEN 1 AND 10
)
WHERE EXISTS (
  SELECT 1 
  FROM RankedProducts
  WHERE RankedProducts.branch_name = branches.branch_name AND ranking BETWEEN 1 AND 10
);''')    

conn.commit() 

In [ ]:
cur.close()
conn.close()